# Single Agent on SummEval 

Please see SOURCES.txt for further source information.

## Imports

In [ ]:
import pandas as pd
from datasets import load_dataset
import os
from dotenv import load_dotenv
from autogen import ConversableAgent
import re
from tqdm import tqdm
import numpy as np

## Data

In [ ]:
# Load and prepare a subset of the SummEval dataset
SummEval_Test = load_dataset("mteb/summeval", split="test")
df = pd.DataFrame(SummEval_Test)
problematic_indices = [5, 7, 8, 9, 10, 11, 18, 20, 26, 27, 33, 34, 39, 46, 61, 64, 68, 73, 75, 79, 85, 86, 88, 92, 96, 99]
df_filtered = df.drop(index=problematic_indices).reset_index(drop=True)
df_filtered = df_filtered[["text", "machine_summaries", "relevance", "coherence", "fluency", "consistency"]]
df_exploded = df_filtered.explode(["machine_summaries", "relevance", "coherence", "fluency", "consistency"]).reset_index(drop=True)
df_sampled = df_exploded.sample(n=100, random_state=42).reset_index(drop=True)
columns_to_round = ["relevance", "coherence", "fluency", "consistency"]
df_sampled[columns_to_round] = df_sampled[columns_to_round].astype(float).round().astype(int)
df_final = df_sampled

print(df_final.info())
print(df_final.head(1))

## Config

In [ ]:
# Load Azure OpenAI configuration from environment variables
load_dotenv()

api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_API_VERSION")

# Start of code citation [1]
# The following code is adapted from the source above.

# Define the model configuration for Azure OpenAI API access
config_list = [
    {
        "model": deployment_name,
        "api_key": api_key,
        "base_url": f"{endpoint}/openai/deployments/{deployment_name}/chat/completions?api-version={api_version}",
        "api_type": "azure",
        "api_version": api_version,  
        "temperature": 0,
        "cache_seed": 42,
    }
]

# End of code citation [1]

## System Design

In [ ]:
# Define the system prompt for the Evaluator-Agent
agent_system_message = f"""
You are an objective Evaluator-Agent.
In this task you will evaluate the quality of a summary written for a news article.
To correctly solve this task, follow these steps:

    1. Carefully read the news article, be aware of the information it contains.
    2. Read the proposed summary.
    3. Rate the summary with integer values on a scale from 1 (worst) to 5 (best) by its relevance, consistency, fluency, and coherence.

Definitions:
    Relevance:
        - The rating measures how well the summary captures the key points of the article.
        - Consider whether all and only the important aspects are contained in the summary.
    Consistency:
        - The rating measures whether the facts in the summary are consistent with the facts in the original article.
        - Consider whether the summary does reproduce all facts accurately and does not make up untrue information.
    Fluency:
        - This rating measures the quality of individual sentences, are they well-written and grammatically correct.
        - Consider the quality of individual sentences.
    Coherence:
        - The rating measures the quality of all sentences collectively, to the fit together and sound naturally.
        - Consider the quality of the summary as a whole.

Give an explanation on your evaluation using about 200 words.
Always begin your output with: "As an objective Evaluator-Agent I think ..."
Always end your output with a JSON object with the following format:{{"relevance": score, "coherence": score, "fluency": score, "consistency": score}} 
"""

In [ ]:
# Start of code citation [1]
# The following code is adapted from the source above.

# Initialize the ConversableAgents for system setup
initializer = ConversableAgent(
    "initializer", 
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    )

agent = ConversableAgent(
    "Evaluator-Agent", 
    llm_config={"config_list": config_list},
    system_message=agent_system_message,
    human_input_mode="NEVER",
    )

# End of code citation [1]

## Evaluation

In [ ]:
# Define the evaluation function
def evaluate(text, machine_summaries, relevance, coherence, fluency, consistency):
    message = f""" 
    Article: {text}

    Summary: {machine_summaries}
    """

    # Start of code citation [1]
    # The following code is adapted from the source above.

    result = initializer.initiate_chat(agent, message=message, max_turns=1)

    # End of code citation [1]

    result_str = str(result)

    pattern = r'"relevance"\s*:\s*(\d+)'
    relevance_score = int(re.search(pattern, result_str).group(1))

    pattern = r'"coherence"\s*:\s*(\d+)'
    coherence_score = int(re.search(pattern, result_str).group(1))

    pattern = r'"fluency"\s*:\s*(\d+)'
    fluency_score = int(re.search(pattern, result_str).group(1))

    pattern = r'"consistency"\s*:\s*(\d+)'
    consistency_score = int(re.search(pattern, result_str).group(1))

    relevance_deviation = relevance_score - relevance
    coherence_deviation = coherence_score - coherence
    fluency_deviation = fluency_score - fluency
    consistency_deviation = consistency_score - consistency

    return {
        "relevance": {
            "ground_truth": relevance,
            "system_decision": relevance_score,
            "deviation": relevance_deviation
        },
        "coherence": {
            "ground_truth": coherence,
            "system_decision": coherence_score,
            "deviation": coherence_deviation
        },
        "fluency": {
            "ground_truth": fluency,
            "system_decision": fluency_score,
            "deviation": fluency_deviation
        },
        "consistency": {
            "ground_truth": consistency,
            "system_decision": consistency_score,
            "deviation": consistency_deviation
        }
    }

In [ ]:
# Prepare evaluation data
num_rows = 100
df_subset = df_final.head(num_rows)

# Evaluate responses
results = []
for _, row in tqdm(df_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        text=row["text"],
        machine_summaries=row["machine_summaries"],
        relevance=row["relevance"],
        coherence=row["coherence"],
        fluency=row["fluency"],
        consistency=row["consistency"]
    )
    results.append(result)

results_df = pd.DataFrame(results)
results_df.to_csv('Results/single.csv', index=False)